In [1]:
import pickle
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
import torch.nn.functional as F
from torch.nn import Linear
import random
import torch.optim as optim

cpnet_path = "data/cpnet.graph"
cpnet = None
cpnet_simple = None

# taken from quentin lol - changed a little
def load_cpnet():
    global cpnet, cpnet_simple
    print("Loading cpnet...")
    with open(cpnet_path, "rb") as f:
        cpnet = pickle.load(f)
    print("Done")

    # Build an undirected version
    cpnet_simple = nx.Graph()
    for u, v, data in cpnet.edges(data=True):
        w = data["weight"] if "weight" in data else 1.0
        if cpnet_simple.has_edge(u, v):
            cpnet_simple[u][v]["weight"] += w
        else:
            cpnet_simple.add_edge(u, v, weight=w)
    
    return cpnet_simple, cpnet.edges(data=True)

# converting to pyG cause its directly usable for models like R-GCN
def convert_to_pyg(cpnet):
    node_map = {node: i for i, node in enumerate(cpnet.nodes())}
    edge_index = []
    edge_attr = []
    rel_map = {}
    
    for u, v, data in cpnet.edges(data=True):
        u_idx, v_idx = node_map[u], node_map[v]
        edge_index.append([u_idx, v_idx])
        rel = data.get("rel", "generic")
        if rel not in rel_map:
            rel_map[rel] = len(rel_map)
        edge_attr.append(rel_map[rel])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.long)
    
    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(node_map)), rel_map

class RGCN(torch.nn.Module):
    def __init__(self, num_nodes, num_relations, in_channels, hidden_channels, out_channels):
        super(RGCN, self).__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels, num_relations)
        self.conv2 = RGCNConv(hidden_channels, out_channels, num_relations)
        self.score_fn = Linear(out_channels * 2, 1)  # Scoring function for link prediction

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        return x
    
    def predict_link(self, x, edge_pairs):
        h1 = x[edge_pairs[:, 0]]
        h2 = x[edge_pairs[:, 1]]
        scores = self.score_fn(torch.cat([h1, h2], dim=1))
        return torch.sigmoid(scores).squeeze()

# apparently the RGCN documentation said you should use negative sampling so this is what that does
def generate_negative_edges(num_nodes, num_samples, existing_edges):
    neg_edges = set()
    while len(neg_edges) < num_samples:
        u, v = random.randint(0, num_nodes - 1), random.randint(0, num_nodes - 1)
        if (u, v) not in existing_edges and (v, u) not in existing_edges and u != v:
            neg_edges.add((u, v))
    return torch.tensor(list(neg_edges), dtype=torch.long)

# train
def train(model, data, x, epochs=100):
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.BCELoss()
    
    pos_edges = data.edge_index.t()
    neg_edges = generate_negative_edges(data.num_nodes, pos_edges.shape[0], set(map(tuple, pos_edges.tolist())))
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        embeddings = model(x, data.edge_index, data.edge_attr)
        pos_scores = model.predict_link(embeddings, pos_edges)
        neg_scores = model.predict_link(embeddings, neg_edges)
        
        labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])])
        scores = torch.cat([pos_scores, neg_scores])
        
        loss = loss_fn(scores, labels)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
# Modify the load_cpnet function to return cpnet_simple
def load_cpnet():
    global cpnet, cpnet_simple
    print("Loading cpnet...")
    with open(cpnet_path, "rb") as f:
        cpnet = pickle.load(f)
    print("Done")

    # Build an undirected version
    cpnet_simple = nx.Graph()
    for u, v, data in cpnet.edges(data=True):
        w = data["weight"] if "weight" in data else 1.0
        if cpnet_simple.has_edge(u, v):
            cpnet_simple[u][v]["weight"] += w
        else:
            cpnet_simple.add_edge(u, v, weight=w)
    
    return cpnet_simple, cpnet.edges(data=True)

c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\torch_geometric\typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(
c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\torch_geometr

In [2]:
# Now call the function and unpack the result
cpnet_simple, edge_types = load_cpnet()

Loading cpnet...
Done


In [3]:
data, rel_map = convert_to_pyg(cpnet_simple)

In [4]:
num_relations = len(rel_map)
num_nodes = data.num_nodes
print(str(num_relations) + ", " + str(num_nodes))

1, 669941


In [5]:
model = RGCN(num_nodes, num_relations, in_channels=64, hidden_channels=128, out_channels=64)

In [6]:
x = torch.randn((num_nodes, 64), dtype=torch.float)

In [7]:
train(model, data, x, epochs=100)

Epoch 0, Loss: 0.7279093265533447
Epoch 10, Loss: 0.6410232782363892
Epoch 20, Loss: 0.4867541491985321
Epoch 30, Loss: 0.3578296899795532
Epoch 40, Loss: 0.3482210338115692
Epoch 50, Loss: 0.3340449333190918
Epoch 60, Loss: 0.3289547562599182
Epoch 70, Loss: 0.3244965672492981
Epoch 80, Loss: 0.3215307593345642
Epoch 90, Loss: 0.31939268112182617


In [10]:
embeddings = model(x, data.edge_index, data.edge_attr).detach()
torch.save(embeddings, "conceptnet_embeddings.pt")

In [11]:
model.eval()
with torch.no_grad():
    embeddings = model(x, data.edge_index, data.edge_attr)
    print("Shape of computed embeddings:", embeddings.shape)  # Should be (num_nodes, out_channels)
    print("Max value in embeddings:", embeddings.max().item())
    print("Min value in embeddings:", embeddings.min().item())
    print("Any NaNs?", torch.isnan(embeddings).any().item())

Shape of computed embeddings: torch.Size([669941, 64])
Max value in embeddings: 4.272341251373291
Min value in embeddings: -4.3894429206848145
Any NaNs? False
